In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp

In [ ]:
#adata.write('/home/julia/data/placodes/h5ad/11_normalized.h5ad')

In [ ]:
adata=sc.read_h5ad('/home/julia/data/placodes/h5ad/11_qc.h5ad')

/home/julia/miniconda3/envs/scenicplus/lib/python3.8/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
#adata=sc.read_h5ad('/home/julia/data/placodes/h5ad/11_normalized.h5ad')

In [ ]:
adata.obs_names_make_unique()

In [ ]:
#adata=sc.read_h5ad('/home/julia/data/placodes/noutbooks/h5ad_files/14_06_ALL_7_datasets_qc.h5ad')

In [ ]:
adata

AnnData object with n_obs × n_vars = 62269 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [ ]:
adata_placodes=sc.read_h5ad('../Epibranchial__TF_UMAP.h5ad')
adata_placodes=adata_placodes.raw.to_adata()

In [ ]:
adata_placodes.obs_names_make_unique()

In [ ]:
cell=adata_placodes.obs_names
adata[cell]

View of AnnData object with n_obs × n_vars = 7676 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [ ]:
adata_new=adata[cell]

In [ ]:
adata_new

View of AnnData object with n_obs × n_vars = 7676 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [ ]:
adata_new.write('adata_Epibranch.h5ad')

In [ ]:
adata=adata_new

In [ ]:
adata

View of AnnData object with n_obs × n_vars = 7676 × 25865
    obs: 'plate', 'day', 'location', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'n_genes', 'doublet_score', 'predicted_doublet', 'batch'
    layers: 'counts'

In [ ]:
# path to unfiltered loom file (this will be created in the optional steps below)
f_loom_path_unfilt = "placods_unfiltered.loom" # test dataset, n=500 cells

# # path to loom file with basic filtering applied (this will be created in the "initial filtering" step below). Optional.
f_loom_path_scenic = "placodes_filtered_scenic.loom"

# path to anndata object, which will be updated to store Scanpy results as they are generated below
#f_anndata_path = "anndata.h5ad"

# path to pyscenic output
f_pyscenic_output = "placodes_pyscenic_output.loom"

# loom output, generated from a combination of Scanpy and pySCENIC results:
f_final_loom = 'placodes_scenic_integrated-output.loom'

In [ ]:
adata.obs_names

Index(['AATGACCAGGAGTACC-1', 'ACCCTCAAGCCACTCG-1', 'ACTTAGGCAAATCCCA-1',
       'ACTTTGTGTCTCGGGT-1', 'AGACAAAGTGACAGGT-1', 'AGTAGCTGTCATGGCC-1',
       'ATCCTATAGATTGATG-1', 'ATGGGTTAGGGTCTTT-1', 'ATTTCACCATGGTGGA-1',
       'CAATACGCAGATCACT-1',
       ...
       'TTTACTGAGGCTAACG-1-3', 'TTTACTGCAAAGACTA-1-3', 'TTTAGTCGTACTCGTA-1-3',
       'TTTCAGTGTCTGTGTA-1-3', 'TTTCAGTGTGGACTGA-1-3', 'TTTCATGTCTCGGCTT-1-3',
       'TTTCCTCAGTACGAGC-1-3', 'TTTCCTCTCATACGAC-1-3', 'TTTCCTCTCCCGTGAG-1-3',
       'TTTGGTTCATGACAAA-1-3'],
      dtype='object', length=7676)

In [ ]:
# create basic row and column attributes for the loom file:
row_attrs = {
    "Gene": np.array(adata.var_names) ,
}
col_attrs = {
    "CellID": np.array(adata.obs_names) ,
    "nGene": np.array( np.sum(adata.X.transpose()>0 , axis=0)).flatten() ,
    "nUMI": np.array( np.sum(adata.X.transpose() , axis=0)).flatten() ,
}
lp.create( f_loom_path_scenic, adata.X.transpose(), row_attrs, col_attrs)

In [ ]:
f_tfs='/home/julia/data/Scenic_database/Mouse/TF/allTFs_mm.txt'

In [ ]:
#!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic grn /home/julia/data/placodes/ANDYs_notebboks/Placodes_tragectory/Epi/Scenic_epi/placodes_filtered_scenic.loom /home/julia/data/Scenic_database/Mouse/TF/allTFs_mm.txt -o adj_1.csv  --num_workers 12


In [ ]:
adjacencies = pd.read_csv("adj_1.csv", index_col=False)

In [ ]:
adjacencies.head()

,TF,target,importance
0,Hmgb2,Ube2c,166.695799
1,Id2,Id1,165.367390
2,Hmgb2,Cks2,159.481001
3,Hmgb2,Cenpa,155.945171
4,Rps4x,Eef1a1,152.874780


In [ ]:
import glob
# ranking databases
f_db_glob = "/home/julia/data/Scenic_database/Mouse/Gene_based/*feather"
f_db_names = ' '.join( glob.glob(f_db_glob) )

In [ ]:
# motif databases
f_motif_path = "/home/julia/data/Scenic_database/Mouse/Motif2TF/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl"

In [ ]:
!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic ctx adj_1.csv \
    {f_db_names} \
    --annotations_fname /home/julia/data/Scenic_database/Mouse/Motif2TF/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl \
    --expression_mtx_fname /home/julia/data/placodes/ANDYs_notebboks/Placodes_tragectory/Epi/Scenic_epi/placodes_filtered_scenic.loom \
    --output reg_EPI.csv \
    --mask_dropouts \
    --num_workers 25


2025-08-26 09:56:20,789 - pyscenic.cli.pyscenic - INFO - Creating modules.

2025-08-26 09:56:23,544 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-08-26 09:56:26,772 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-08-26 09:56:28,273 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-08-26 09:57:12,953 - pyscenic.utils - INFO - Creating modules.

2025-08-26 09:59:46,273 - pyscenic.cli.pyscenic - IN

In [ ]:
!/home/julia/miniconda3/envs/scenicplus/bin/pyscenic aucell \
    /home/julia/data/placodes/ANDYs_notebboks/Placodes_tragectory/Epi/Scenic_epi/placodes_filtered_scenic.loom \
    /home/julia/data/placodes/ANDYs_notebboks/Placodes_tragectory/Epi/Scenic_epi/reg_EPI.csv \
    --output /home/julia/data/placodes/ANDYs_notebboks/Placodes_tragectory/Epi/Scenic_epi/placodes_pyscenic_output.loom \
    --num_workers 25


2025-08-26 10:40:12,370 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-08-26 10:40:15,266 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2025-08-26 10:40:18,960 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2025-08-26 10:40:42,550 - pyscenic.cli.pyscenic - INFO - Writing results to file.
/home/julia/miniconda3/envs/scenicplus/lib/python3.8/site-packages/pyscenic/cli/utils.py:347: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, threshold in auc_thresholds.iteritems()
